#### Setup environment

In [1]:
# %load "/Users/haleighwright/Desktop/NCEAS/pyriv/notebooks/nb_config.py"
import sys
sys.path.append("/Users/haleighwright/Desktop/NCEAS/pyriv")
sys.path.append("/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages")
import networkx as nx
import numpy as np
import sys
import geopandas as gpd
from shapely.geometry import Point
from multiprocessing import Pool

from pyriv import graph_prep as GraphBuilder
from pyriv import river_graph as RiverGraph

%matplotlib inline

In [2]:
from functools import partial
from types import *

### Retrieve the dead ends of the network
The dead ends are the nodes that have an out degree = 0

In [3]:
# to be efficient with this process, we need to assign node ID's - nodes are identified by their coord's when read in

In [4]:
def add_node_ids_copy(G):
    """
    Params: networkx graph
    Return: copy of graph with node ID attr. added
    
    Returns a copy of the graph with node ID attr (an int) added, 
    used for finding problem nodes easier / quicker node identification
    Must use a dict with the nodes' as keys and the attributes listed
    """
    copy = G.copy()
    return add_node_ids_mutate(copy)

In [5]:
def node_to_dict(G):
    nodes = nx.nodes(G)
    #check if they have id first?
    #if nodes[0]
    temp = {}
    i = 0
    for node in nodes:
        temp[i] = node
        i = i+1
        
    return temp

In [6]:
def add_node_ids_mutate(G):
    """
    Params: networkx graph
    Return: referfence to modified graph
    
    Modifies input graph with node ID attr (an int) added, 
    used for finding problem nodes easier / quicker node identification
    Must use a dict with the nodes' as keys and the attributes listed
    """
    attr_dict = {}
    i = 0
    keys = G.nodes()
    for node in keys:
        attr_dict[node] = i
        i = i+1
    nx.set_node_attributes(G, 'ID', attr_dict)
    return G

In [7]:
def deadends_old(self):
    """
    For a directed graph, find the nodes that have out_degree == 0
    Note: this is the older, slower method
    """
    thedict = {}
    # get a dictionary of { node: out_degree }
    degdict = self.out_degree(self.nodes())
    # convert to a 2 column array of node and out_degree
    degarr = np.array(degdict.items())
    # get a boolean index of rows where out_degree==0
    dead_row_ind = (degarr[:,1]==0)

    
    for i in range(0, len(degarr)):
        if dead_row_ind[i] == True:
            thedict[allnodes[i][0]] = tuple(degarr[:,0][i])
            
    # use that index to get the nodes
    #dead_nodes = degarr[:,0][dead_row_ind]
    return thedict

In [8]:
def deadend_coords_to_keys(G, ends):
    assert type(ends) is TupleType, "collection of end nodes is not a tuple"
    ends = list(ends)
    
    
    alldict = nx.get_node_attributes(G, 'ID')
    
    keylist = []
    for i in range(0, len(ends)):
        keylist.append(alldict[ends[i]])
        
    return keylist
        
    

In [9]:
def find_missing_edges_seq(G, ends, allnodes, threshold):
    """
    Params:
    Returns: list of missing edges

    Sequential method to find missing edges. VERY VERY SLOW; use ||'zd version
    """
    for key1, node1 in deadends.iteritems():
        for key2, node2 in allnodes.iteritems():
            #print node1
            xd = node1[0] - node2[0]
            yd = node1[1] - node2[1]
            dist = np.sqrt((xd*xd + yd*yd))
            
            if key1 != key2 and dist < threshold and dist > 0:
                graph.add_edge(key1, key2)
                print("added edge from "+str(key1)+" : "+str(node1)+" to "+ str(key2)+" : "+str(node2))
                problem_points[str(cnt)+"_A"] =key1
                problem_points[str(cnt)+"_B"] =key2
                break
                
        



In [51]:
def find_missing_edges_par(ends, nodes, threshold, numproc):
    """
    Params: ends = deadends found dict, allnodes = iterable of all nodes; threshold = dist. thresh
    Returns: a list of missing edges to add where dist < thresh. b/w the deadend & other node
    
    ||'zd method to find missing edges (a CPU-bound task w/ no communication needs)
    
    Note 1: in order to pass multiple args to multiproc.'g map fcn. we use a partial 
                                                                ( req. Python >=2.7)
    Note 2: Pool.map maps a fcn over sequence (str, unicode, list, tuple, buffer, or xrange) so
                        need to convert dict to that first
    """
    assert type(ends) is ListType, "collection of end nodes is not a list"
    assert type(ends[0]) is IntType, "nodes in ends list arent referred to by their ID"
    
    assert type(nodes) is DictType, "collection of allnodes is not a dictionary"
    
    #need to convert the dict of dead ends to the correct format, which is:
    #    key: (key, (coord_x, coord_y))

    realdict = {}
    for key, val in nodes.iteritems():
        realdict[key]=(key, val)
        
    #assert type(allnodes[0]) is IntType, "first column of allnodes dictionary is not a node ID"
    #assert type(allnodes[0][0]) is TupleType, "second column of allnodes dictionary is not a coord tuple"
    
    
    if numproc < 2:
        print("Please request to run more than one concurrent process to avoid redundant work. If you wish to use "+
                +"the sequential version of this method, please do so directly.")
    else:
        
        
          
        pool = Pool(numproc)
    
        result_list = pool.map(partial(find_missing_edges_par_inner, allnodes=realdict, th=threshold), ends)
        pool.close()
        pool.join()
        
        cleaned_results = []
        #clean result list before returning or else it will have empty entries
        for entry in result_list:
            if not not entry:
                if len(entry) > 1:
                    for innerentry in entry:
                        cleaned_results.append(innerentry)
                else:
                    cleaned_results.append(entry[0])
                
        return cleaned_results 
    
    

In [10]:
def find_missing_edges_par_inner(key1, allnodes, th):
    """
    inner function for || version of finding missing edges
    """
    
    edges_to_add = []
    node1 = allnodes[key1][1]
    for i in range(0, len(allnodes)):
        key2 = allnodes[i][0]
        if key1 != key2:
            node2 = allnodes[i][1]
            xd = node1[0] - node2[0]
            yd = node1[1] - node2[1]
            dist = np.sqrt((xd*xd + yd*yd))
            if key1 != key2 and dist < th and dist > 0:
                edges_to_add.append([key1,key2])
                break
                
    return edges_to_add
    
    
    

In [70]:
def add_missing_edges(G, edges_to_add, allnodes):
    """
    Params: G = graph to add edges to; list of edges to add w/ each entry an [x,y] list
    Returns: nothing
    
    IMPORTANT: MUTATES the graph passed in
    """
    for coord in edges_to_add:
        G.add_edge(allnodes[coord[0]], allnodes[coord[1]])

In [13]:
def missing_edges_to_shp(pts, filepath, projection_code):
    """
    Params: nodes/pts describing missing edges; relative filepath .shp to write to; projection_code to convert
            coords (should match the maps you started with for overlaying purposes)
    Returns: none (writes to output file) 
    
    
    """
    pts = list(pts.values())
    kcdf = gpd.GeoDataFrame({'geometry': [Point(n) for n in pts]})
    kcdf.crs = {'init': projection_code}
    kcdf.to_file(filepath)

In [14]:
def component_stats(G, verbose):
    """
    Params: G = graph to collect stats on, verbose flag
    Returns: nothing
    
    Prints out various relevent stats about graphs concerning components
    """
    explans = {}
    if verbose == True:
        explans['weakly-connected'] = "(There is an undirected path between each pair of nodes in the directed graph)"
        explans['strongly-connected'] = "(There is a directed path between each pair of nodes in the directed graph)"
        explans['semiconnected'] = ""
    else:
        explans['weakly-connected'] = ""
        explans['strongly-connected'] = ""
        explans['semiconnected'] = ""
        
    
    print "Is the graph weakly connected "+explans['weakly-connected'] +"? "+ str(nx.is_weakly_connected(G)) 
    print "Number of weakly connected components: " + str(nx.number_weakly_connected_components(G))
    print "Is the graph semiconnected "+explans['semiconnected']+ "? " + str(nx.is_semiconnected(G))
    print "Is the graph strongly connected "+explans['strongly-connected']+ "? "+ str(nx.is_strongly_connected(G))
    
    
    
    

In [15]:
def general_stats(G):
    """
    Print general stats about graph G
    Can add more as/if needed...
    """
    print "Number of edges: "+ str(G.number_of_edges())
    print "Number of nodes: "+str(G.number_of_nodes()) 

# Test The Methods
Go through and test the above methods to assure they all work properly

In [16]:
kusko_g = GraphBuilder.GraphBuilder('../../data/sasap/kusko_region_flowlines.gpickle')

Weighting Edges with Distances


In [17]:
kusko_g_ID = add_node_ids_copy(kusko_g.graph)

In [26]:
component_stats(kusko_g_ID, True)

Is the graph weakly connected (There is an undirected path between each pair of nodes in the directed graph)? False
Number of weakly connected components: 5548
Is the graph semiconnected ? False
Is the graph strongly connected (There is a directed path between each pair of nodes in the directed graph)? False


In [27]:
general_stats(kusko_g_ID)

Number of edges: 284697
Number of nodes: 277425


In [18]:
kusko_ends = kusko_g_ID.deadends()

In [19]:
print kusko_ends[0]

(-471290.9307224038, 1213112.07627931)


In [20]:
print type(kusko_ends)


<type 'tuple'>


In [21]:
kusko_ends_list = deadend_coords_to_keys(kusko_g_ID, kusko_ends)

In [22]:
print type(kusko_ends_list)
print kusko_ends_list[0]

<type 'list'>
13


In [23]:
allnodesdict = node_to_dict(kusko_g_ID)


In [26]:
missing_edges = find_missing_edges_par(kusko_ends_list, allnodesdict, 0.3, 8)

In [50]:
cleaned_results = []
#clean result list before returning or else it will have empty entries
#added this to the missing_edges function directly
for entry in missing_edges:
            if not not entry:
                if len(entry) > 1:
                    for innerentry in entry:
                        cleaned_results.append(innerentry)
                else:
                    cleaned_results.append(entry[0])
                

In [52]:
print cleaned_results

[[203007, 246103]]


In [57]:
copy_kusko2 = kusko_g_ID.copy()

In [56]:
allnodesdict[203007]

(-290938.300311207, 1301283.3149657596)

In [68]:
add_missing_edges(copy_kusko2, cleaned_results, allnodesdict)

203007
246103
(-290938.300311207, 1301283.3149657596)
(-290938.32268009393, 1301283.3127322474)


In [69]:
general_stats(copy_kusko2)

Number of edges: 284698
Number of nodes: 277425
